In [ ]:
%load_ext autoreload
%autoreload 2



In [1]:
# setup
import ee, geemap, pandas as pd, geopandas as gpd
from shapely.geometry import Point

ee.Authenticate()
ee.Initialize()


ModuleNotFoundError: No module named 'geopandas'

- wpdx data - base

In [ ]:
# Load and Sample WPdx
import re

wpdx = pd.read_csv('C:/Users/ADMIN/Documents/MoringaSchool/Phase5CapstoneProject/data/wpdx_kenya.csv', low_memory=False)

wpdx.head()


,row_id,#source,#lat_deg,#lon_deg,#report_date,#status_id,#water_source_clean,#water_source_category,#water_tech_clean,#water_tech_category,...,#subjective_quality,New Georeferenced Column,lat_lon_deg,public_data_source,converted,count,created_timestamp,updated_timestamp,is_duplicate,dataset_title
0,"5,426",COWASH,4.293253,41.139771,2020 May 14 12:00:00 AM,Yes,Protected Well,Well,NaN,NaN,...,NaN,POINT (41.139771 4.293253),"(4.293253°, 41.139771°)",https://catalog.waterpointdata.org/datafiles/c...,NaN,1,2021 Feb 19 12:03:25 AM,2021 Feb 19 12:03:25 AM,NaN,COWASH_Ethiopia_2018-2021
1,"770,337",COWASH,11.533516,37.968994,2021 Jan 01 12:00:00 AM,Yes,Protected Well,Well,NaN,NaN,...,NaN,POINT (37.968994 11.533516),"(11.533516°, 37.968994°)",https://catalog.waterpointdata.org/datafiles/c...,NaN,1,2021 Feb 19 12:03:25 AM,2021 Feb 19 12:03:25 AM,NaN,COWASH_Ethiopia_2018-2021
2,"792,314",COWASH,11.247572,37.547421,2019 Oct 28 12:00:00 AM,Yes,Protected Well,Well,NaN,NaN,...,NaN,POINT (37.547421 11.247572),"(11.247572°, 37.547421°)",https://catalog.waterpointdata.org/datafiles/c...,NaN,1,2021 Feb 19 12:03:25 AM,2021 Feb 19 12:03:25 AM,NaN,COWASH_Ethiopia_2018-2021
3,"770,248",COWASH,11.263851,37.976189,2021 Jan 01 12:00:00 AM,Yes,Protected Well,Well,NaN,NaN,...,NaN,POINT (37.976189 11.263851),"(11.263851°, 37.976189°)",https://catalog.waterpointdata.org/datafiles/c...,NaN,1,2021 Feb 19 12:03:25 AM,2021 Feb 19 12:03:25 AM,NaN,COWASH_Ethiopia_2018-2021
4,"770,247",COWASH,11.268485,37.987934,2021 Jan 01 12:00:00 AM,Yes,Protected Well,Well,NaN,NaN,...,NaN,POINT (37.987934 11.268485),"(11.268485°, 37.987934°)",https://catalog.waterpointdata.org/datafiles/c...,NaN,1,2021 Feb 19 12:03:25 AM,2021 Feb 19 12:03:25 AM,NaN,COWASH_Ethiopia_2018-2021


In [ ]:
wpdx.columns 

Index(['row_id', '#source', '#lat_deg', '#lon_deg', '#report_date',
       '#status_id', '#water_source_clean', '#water_source_category',
       '#water_tech_clean', '#water_tech_category', '#facility_type',
       '#clean_country_name', '#clean_country_id', '#clean_adm1',
       '#clean_adm2', '#clean_adm3', '#clean_adm4', '#install_year',
       '#installer', '#rehab_year', '#rehabilitator', '#management_clean',
       '#status_clean', '#pay_clean', '#fecal_coliform_presence',
       '#fecal_coliform_value', '#subjective_quality_clean', '#activity_id',
       '#scheme_id', '#wpdx_id', '#notes', '#orig_lnk', '#photo_lnk',
       '#country_id', '#data_lnk', '#distance_to_primary_road',
       '#distance_to_secondary_road', '#distance_to_tertiary_road',
       '#distance_to_city', '#distance_to_town', 'water_point_history',
       'local_population_1km', 'water_point_population', 'rehab_priority',
       '#pop_who_would_gain_access', 'crucialness_score', 'pressure_score',
       'usage_

In [ ]:
# --- Standardize and Rename Columns ---
# No need for the full generic column cleaning with re.sub if you know the exact names.
# However, for robustness, we can still ensure they are lowercased and stripped IF needed.
# For now, let's target the exact names you provided.

# Rename 'wpdx_id' to 'pt_id'
if '#wpdx_id' in wpdx.columns:
    wpdx.rename(columns={'#wpdx_id': 'pt_id'}, inplace=True)
else:
    # If for some reason 'wpdx_id' is missing, you'd need to identify the correct ID column.
    raise KeyError("'wpdx_id' column not found. Please verify the actual ID column name.")

# Ensure 'pt_id' is string type
wpdx['pt_id'] = wpdx['pt_id'].astype(str)

# Ensure 'lat_deg' and 'lon_deg' are numeric and rounded
# They are already correctly named, just need conversion and null check.
wpdx['latitude'] = pd.to_numeric(wpdx['#lat_deg'], errors='coerce').round(6)
wpdx['longitude'] = pd.to_numeric(wpdx['#lon_deg'], errors='coerce').round(6)
wpdx.dropna(subset=['latitude', 'longitude'], inplace=True)


# Filter for Kenya and non-null lat/lon
# Using 'clean_country_name' as per your original intent and its presence.
if '#clean_country_name' in wpdx.columns:
    wpdx = wpdx[(wpdx['#clean_country_name'].str.lower() == 'kenya')]
else:
    raise KeyError("'country_name' column not found. Please verify the country column name.")

# Sample 5000 points
DESIRED_SAMPLED_POINTS_COUNT = 5000
if len(wpdx) >= DESIRED_SAMPLED_POINTS_COUNT:
    wpdx = wpdx.sample(n=DESIRED_SAMPLED_POINTS_COUNT, random_state=42).reset_index(drop=True)
else:
    print(f"Warning: Less than {DESIRED_SAMPLED_POINTS_COUNT} points available for sampling. Using all {len(wpdx)} points.")

print(f"Sampled {len(wpdx)} water points for GEE processing.")

# --- Prepare for GEE (select only necessary columns with correct names for geemap) ---
wpdx_for_gee = wpdx[['pt_id', 'latitude', 'longitude']]
# Note: geemap.pandas_to_ee expects 'latitude' and 'longitude' by default if no other names specified.
# So, we map lat_deg/lon_deg to latitude/longitude for clean passing.

Sampled 5000 water points for GEE processing.


In [ ]:
wpdx_for_gee.head()

,pt_id,latitude,longitude
0,6GFP7JP2+45J,-0.714673,34.600380
1,6GFPRRXX+7Q5,-0.151875,34.849491
2,6GFPWC7Q+RFP,-0.085423,34.438709
3,6GFVGH59+525,-0.492120,37.567619
4,6GGP6Q8Q+567,0.215407,34.788044


In [ ]:
# Set the END date to June 30th, 2025 
END_DATE = '2025-06-30' 

# Recalculate start dates based on the new END date
START_7_DAYS  = ee.Date(END_DATE).advance(-7, 'day').format('YYYY-MM-dd').getInfo()
START_30_DAYS = ee.Date(END_DATE).advance(-30, 'day').format('YYYY-MM-dd').getInfo()
START_90_DAYS = ee.Date(END_DATE).advance(-90, 'day').format('YYYY-MM-dd').getInfo()

print(f"Date range for composites: {START_90_DAYS} to {END_DATE}")
print(f"Calculated START_7: {START_7_DAYS}")
print(f"Calculated START_30: {START_30_DAYS}")
print(f"Calculated START_90: {START_90_DAYS}")

Date range for composites: 2025-04-01 to 2025-06-30
Calculated START_7: 2025-06-23
Calculated START_30: 2025-05-31
Calculated START_90: 2025-04-01


In [ ]:
# Define the image collections
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").select("precipitation")
ndvi = ee.ImageCollection("MODIS/061/MOD13Q1").select("NDVI")
lst = ee.ImageCollection("MODIS/061/MOD11A2").select("LST_Day_1km")
soil = ee.ImageCollection("NASA_USDA/HSL/SMAP_soil_moisture").select("ssm")
pop = ee.Image("WorldPop/GP/100m/pop/KEN_2020").select("population") # 'pop' band in WorldPop 100m dataset is 'population'

c:\Users\ADMIN\anaconda3\envs\learn-env\lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for NASA_USDA/HSL/SMAP_soil_moisture! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_SMAP_soil_moisture

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
# Convert pandas DataFrame to ee.FeatureCollection
wpdx_fc = geemap.pandas_to_ee(wpdx_for_gee, latitude='latitude', longitude='longitude', id_field='pt_id')

print(f"Converted {wpdx_fc.size().getInfo()} water points to ee.FeatureCollection.")

Converted 5000 water points to ee.FeatureCollection.


In [ ]:
# Function for Extraction 


def extract_var(ic_or_img, start, end, band, scale):
    """
    Extracts mean value of a band for each feature in wpdx_fc from an ImageCollection or Image.
    Applies scale factors and handles static images.
    """
    if isinstance(ic_or_img, ee.ImageCollection):
        img = ic_or_img.filterDate(start, end).mean().select(band)
    else: # Assume it's an ee.Image for static data
        img = ic_or_img.select(band)

    # Apply specific scale factors/conversions
    if band == "NDVI":
        img = img.multiply(0.0001) # MODIS NDVI scale factor
    elif band == "LST_Day_1km":
        img = img.multiply(0.02).subtract(273.15) # MODIS LST scale factor (K to C)

    # Sample regions and return as FeatureCollection
    return img.sampleRegions(collection=wpdx_fc, scale=scale, geometries=True)

print("Extraction function defined.")


Extraction function defined.


In [ ]:
# --- 4. Apply for All Variables ---

# CHIRPS (Daily Precipitation)
chirps_7_fc = extract_var(chirps, START_7_DAYS, END_DATE, "precipitation", 5000)
chirps_30_fc = extract_var(chirps, START_30_DAYS, END_DATE, "precipitation", 5000)
chirps_90_fc = extract_var(chirps, START_90_DAYS, END_DATE, "precipitation", 5000)

# NDVI (16-day composite)
ndvi_7_fc = extract_var(ndvi, START_7_DAYS, END_DATE, "NDVI", 250)
ndvi_30_fc = extract_var(ndvi, START_30_DAYS, END_DATE, "NDVI", 250)
ndvi_90_fc = extract_var(ndvi, START_90_DAYS, END_DATE, "NDVI", 250)

# LST (8-day composite)
lst_7_fc = extract_var(lst, START_7_DAYS, END_DATE, "LST_Day_1km", 1000)
lst_30_fc = extract_var(lst, START_30_DAYS, END_DATE, "LST_Day_1km", 1000)
lst_90_fc = extract_var(lst, START_90_DAYS, END_DATE, "LST_Day_1km", 1000)

# Soil Moisture (Daily)
soil_7_fc = extract_var(soil, START_7_DAYS, END_DATE, "ssm", 9000)
soil_30_fc = extract_var(soil, START_30_DAYS, END_DATE, "ssm", 9000)
soil_90_fc = extract_var(soil, START_90_DAYS, END_DATE, "ssm", 9000)

# Population Density (Static Image)
pop_feat_fc = extract_var(pop, None, None, "population", 100)

print("Environmental FeatureCollections generated.")

Environmental FeatureCollections generated.


In [ ]:
# --- 5. Example of Accessing Data (Optional) ---
try:
    print("\n--- Example Retrieved Data (first feature) ---")
    print(f"CHIRPS 30-day (precipitation): {chirps_30_fc.first().getInfo().get('properties')}")
    print(f"NDVI 30-day (NDVI): {ndvi_30_fc.first().getInfo().get('properties')}")
    print(f"LST 30-day (LST_Day_1km): {lst_30_fc.first().getInfo().get('properties')}")
    print(f"Soil 30-day (ssm): {soil_30_fc.first().getInfo().get('properties')}")
    print(f"Population (population): {pop_feat_fc.first().getInfo().get('properties')}")
except Exception as e:
    print(f"\nCould not retrieve example data (might be empty FeatureCollection or other GEE error): {e}")



--- Example Retrieved Data (first feature) ---
CHIRPS 30-day (precipitation): {'latitude': -0.714673, 'longitude': 34.60038, 'precipitation': 2.6976897716522217, 'pt_id': '6GFP7JP2+45J'}
NDVI 30-day (NDVI): {'NDVI': 0.7704000000000001, 'latitude': -0.714673, 'longitude': 34.60038, 'pt_id': '6GFP7JP2+45J'}
LST 30-day (LST_Day_1km): {'LST_Day_1km': 24.225000000000023, 'latitude': -0.714673, 'longitude': 34.60038, 'pt_id': '6GFP7JP2+45J'}

Could not retrieve example data (might be empty FeatureCollection or other GEE error): Image.select: Band pattern 'ssm' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands


- the soil one needs to be removed maybe

In [ ]:
# --- 5. Example of Accessing Data (Optional) ---
try:
    print("\n--- Example Retrieved Data (first feature) ---")
    print(f"CHIRPS 30-day (precipitation): {chirps_30_fc.first().getInfo().get('properties')}")
    print(f"NDVI 30-day (NDVI): {ndvi_30_fc.first().getInfo().get('properties')}")
    print(f"LST 30-day (LST_Day_1km): {lst_30_fc.first().getInfo().get('properties')}")
    print(f"Population (population): {pop_feat_fc.first().getInfo().get('properties')}")
except Exception as e:
    print(f"\nCould not retrieve example data (might be empty FeatureCollection or other GEE error): {e}")



--- Example Retrieved Data (first feature) ---
CHIRPS 30-day (precipitation): {'latitude': -0.714673, 'longitude': 34.60038, 'precipitation': 2.6976897716522217, 'pt_id': '6GFP7JP2+45J'}
NDVI 30-day (NDVI): {'NDVI': 0.7704000000000001, 'latitude': -0.714673, 'longitude': 34.60038, 'pt_id': '6GFP7JP2+45J'}
LST 30-day (LST_Day_1km): {'LST_Day_1km': 24.225000000000023, 'latitude': -0.714673, 'longitude': 34.60038, 'pt_id': '6GFP7JP2+45J'}
Population (population): {'latitude': -0.714673, 'longitude': 34.60038, 'population': 4.2701640129089355, 'pt_id': '6GFP7JP2+45J'}


## Convert to Pandas Dataframes

In [ ]:
from datetime import datetime

df_chirps_7 = geemap.ee_to_df(chirps_7_fc)
df_chirps_30 = geemap.ee_to_df(chirps_30_fc)
df_chirps_90 = geemap.ee_to_df(chirps_90_fc)

df_ndvi_7 = geemap.ee_to_df(ndvi_7_fc)
df_ndvi_30 = geemap.ee_to_df(ndvi_30_fc)
df_ndvi_90 = geemap.ee_to_df(ndvi_90_fc)

df_lst_7 = geemap.ee_to_df(lst_7_fc)
df_lst_30 = geemap.ee_to_df(lst_30_fc)
df_lst_90 = geemap.ee_to_df(lst_90_fc)

df_pop = geemap.ee_to_df(pop_feat_fc)

final_df = wpdx.copy()

def merge_ee_df(main_df, ee_df, new_col_prefix):
    data_cols = [col for col in ee_df.columns if col not in ['pt_id', 'system:index', '.geo']]
    if not data_cols:
        return main_df
    extracted_col_name = data_cols[0]
    ee_df_renamed = ee_df[['pt_id', extracted_col_name]].rename(columns={extracted_col_name: f'{new_col_prefix}_{extracted_col_name}'})
    return pd.merge(main_df, ee_df_renamed, on='pt_id', how='left')

final_df = merge_ee_df(final_df, df_chirps_7, 'chirps_7')
final_df = merge_ee_df(final_df, df_chirps_30, 'chirps_30')
final_df = merge_ee_df(final_df, df_chirps_90, 'chirps_90')

final_df = merge_ee_df(final_df, df_ndvi_7, 'ndvi_7')
final_df = merge_ee_df(final_df, df_ndvi_30, 'ndvi_30')
final_df = merge_ee_df(final_df, df_ndvi_90, 'ndvi_90')

final_df = merge_ee_df(final_df, df_lst_7, 'lst_7')
final_df = merge_ee_df(final_df, df_lst_30, 'lst_30')
final_df = merge_ee_df(final_df, df_lst_90, 'lst_90')

final_df = merge_ee_df(final_df, df_pop, 'pop')

output_csv_filename = f"kenya_waterpoints_environmental_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
final_df.to_csv(output_csv_filename, index=False)

In [ ]:
final_df.head()

,row_id,#source,#lat_deg,#lon_deg,#report_date,#status_id,#water_source_clean,#water_source_category,#water_tech_clean,#water_tech_category,...,chirps_7_latitude,chirps_30_latitude,chirps_90_latitude,ndvi_7_NDVI,ndvi_30_NDVI,ndvi_90_NDVI,lst_7_LST_Day_1km,lst_30_LST_Day_1km,lst_90_LST_Day_1km,pop_latitude
0,"212,437",Evidence Action,-0.714673,34.600380,2024 Jan 26 12:00:00 AM,Yes,Protected Well,Well,Motorized Pump - Electric,Motorized Pump,...,-0.714673,-0.714673,-0.714673,0.7638,0.77040,0.766650,24.25,24.225000,25.044545,-0.714673
1,"350,174",Virtual Kenya,-0.151875,34.849491,2012 May 28 12:00:00 AM,Yes,Protected Well,Well,Hand Pump - Afridev,Hand Pump,...,-0.151875,-0.151875,-0.151875,0.5566,0.60245,0.677600,27.73,27.035000,26.577273,-0.151875
2,"351,062",Virtual Kenya,-0.085423,34.438709,2012 Jun 05 12:00:00 AM,Yes,Protected Well,Well,Hand Pump - India Mark,Hand Pump,...,-0.085423,-0.085423,-0.085423,0.7461,0.75990,0.774750,25.89,24.480000,24.628000,-0.085423
3,"349,725",Virtual Kenya,-0.492120,37.567619,2012 Dec 16 12:00:00 AM,Yes,Rainwater Harvesting,Rainwater Harvesting,Rope and Bucket,Rope and Bucket,...,-0.492120,-0.492120,-0.492120,0.2576,0.49740,0.705683,NaN,22.816667,22.558000,-0.492120
4,"173,351",Evidence Action,0.215407,34.788044,2023 Oct 11 12:00:00 AM,Yes,Protected Well,Well,Motorized Pump - Electric,Motorized Pump,...,0.215407,0.215407,0.215407,0.7795,0.78995,0.774667,23.07,22.840000,23.131818,0.215407


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 85 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   row_id                       5000 non-null   object 
 1   #source                      5000 non-null   object 
 2   #lat_deg                     5000 non-null   float64
 3   #lon_deg                     5000 non-null   float64
 4   #report_date                 5000 non-null   object 
 5   #status_id                   5000 non-null   object 
 6   #water_source_clean          4967 non-null   object 
 7   #water_source_category       4967 non-null   object 
 8   #water_tech_clean            3679 non-null   object 
 9   #water_tech_category         3518 non-null   object 
 10  #facility_type               5000 non-null   object 
 11  #clean_country_name          5000 non-null   object 
 12  #clean_country_id            5000 non-null   object 
 13  #clean_adm1       